In [25]:
#This is an implementation of the first algorithm shown in the paper : clustering with generalized eigen-decomposition.

# Fichier à supprimer

In [26]:
import numpy as np
from sklearn.cluster import KMeans
from scipy.optimize import minimize
import warnings
warnings.filterwarnings("ignore")
from utils.utils import *
from utils.GraphCreation import graphCreation

In [30]:
#variant : we solve the total problem with respect to both variables

#alpha and beta have to be choosen according to the dataset we are studying

alpha = 10
beta = 100

#Meilame: On le garde ou pas ce truc ?
def spectral_clustering_variant(W,k):
  n,_,M = W.shape
  L = np.zeros(np.shape(W))
  for i in range(M):
    D = computeDegreeMatrix(W[:, :, i])
    L[:, :, i] = np.linalg.inv(D)@(D-W[:,:, i])

  def function_to_optimize(vector):
    P_vectorized = vector[:n*n]
    Q_vectorized = vector[n*n:]
    P = P_vectorized.reshape((n,n))
    Q = Q_vectorized.reshape((n,n))
    res = 0.5*alpha*(np.linalg.norm(P,'fro') + np.linalg.norm(Q,'fro')) + 0.5*beta*np.linalg.norm(P@Q-np.identity(n),'fro')
    for i in range(M):
      eigenvalues,eigenvectors = np.linalg.eig(L[:, :, i])
      delta = np.diag(eigenvalues)
      res += 0.5*np.linalg.norm(L[:, :, i] - P@delta@Q,'fro')
    return res

  eigenvalues,eigenvectors = np.linalg.eig(L[: ,:, 0])
  x0 = np.concatenate((eigenvectors.flatten(),np.linalg.inv(eigenvectors).flatten()))
  result_optimization = minimize(function_to_optimize, x0,method='L-BFGS-B')

  print("Optimisation réussie : ", result_optimization.success)
  print(result_optimization.message)

  result_optimization = result_optimization.x

  U = result_optimization[:n*n].reshape((n,n))
  U = U[:,:k]
  return k_means_cluster(U,k)

In [31]:
spectral_clustering_variant(matrix,k=5)

KeyboardInterrupt: 